In [36]:
import tkinter as tk
import subprocess
import os
import time
from tkinter import ttk, messagebox
import logging # SET UP LOGGING

class MyApplication(tk.Tk):
    def __init__(self):
        self.OMNI = 'Omni_stage'
        self.IMED = 'Imed_stage'
        super().__init__()
        self.title("Or-Q")
        self.geometry("600x400")

        # -------------------------------------------- STYLE ---------------------------------------------
        
        style = ttk.Style(self)
        style.theme_use("alt")
        style.configure("TButton", font=("Montserrat", 8), padding=5)
        style.configure("Red.TButton", background="red", foreground="black", font=("Montserrat", 8))
        style.configure("Green.TButton", background="green", foreground="black", font=("Montserrat", 8))

        style.configure("TLabel", font=("Montserrat", 8), foreground="black", background="#f0f0f0")
        style.configure("Red.TLabel", foreground="red", font=("Montserrat", 8), background="#f0f0f0")
        style.configure("Green.TLabel",foreground="green", font=("Montserrat", 8), background="#f0f0f0")
        style.configure("Grey.TLabel",foreground="grey", font=("Montserrat", 8), background="#f0f0f0")
                
        style.map("TButton",background=[("active", "lightgrey"), ("pressed", "lightgrey")],
          foreground=[("active", "grey"), ("pressed", "grey")])
        
        style.map("Green.TButton",background=[("active", "lightgreen"), ("pressed", "lightgreen")],
          foreground=[("active", "green"), ("pressed", "green")])

        style.map("Red.TButton",background=[("active", "pink"), ("pressed", "pink")],
          foreground=[("active", "red"), ("pressed", "red")])
        
        # -------------------------------------------- DEFINE VARIABLES ---------------------------------------------
        
        # Boolean variables
        self.omni_running  = tk.BooleanVar(value = False) # status of OMNI: running?
        self.imed_running  = tk.BooleanVar(value = False) # status of IMED: running?
        self.variables_set = tk.BooleanVar(value = False) # required datapoints are defined?
        self.process_stop  = tk.BooleanVar(value = False) # required datapoints are defined?
        
        # Integer variables
        self.omni_run_count = tk.IntVar(value = 0) # count the number of omni runs
        self.imed_run_count = tk.IntVar(value = 0) # count the number of IMED runs
        self.initial_exp_count = tk.IntVar(value = 0)

        # String variables
        self.status_message = tk.StringVar(value='') 
        self.error_message = tk.StringVar(value='') 
        self.variables_set_message = tk.StringVar(value='')
        
        self.omni_text   = tk.StringVar(value = "OMNIBUS")
        self.imed_text   = tk.StringVar(value = "IMED")
        self.omni_folder = tk.StringVar(value = "")
        self.imed_folder = tk.StringVar(value = "")
        self.omni_folder_refinement = tk.StringVar(value = "")
        self.imed_folder_refinement = tk.StringVar(value = "")
        
        self.omni_filename = 'expttsd'
        self.imed_filename = 'imed'

        # --------------------------- Create Widgets ----------------------------------
        self.create_labels()
        self.create_buttons()
        self.create_textboxes()
        self.layout_widgets()
        
    # Labels
    def create_labels(self):
        
        self.Omni_label1 = ttk.Label(self,  textvariable=self.omni_text)
        self.Imed_label1 = ttk.Label(self,  textvariable=self.imed_text)
        self.general_label1 = ttk.Label(self, text = 'Currently running:')
        self.general_label2 = ttk.Label(self, text = 'No. of runs:')
        self.general_label3 = ttk.Label(self, textvariable = self.status_message)
        self.general_label4 = ttk.Label(self, textvariable = self.error_message)
        self.general_label5 = ttk.Label(self, textvariable = self.variables_set_message)
        self.Omni_label2 = ttk.Label(self, text=str(self.omni_run_count.get()))
        self.Imed_label2 = ttk.Label(self, text=str(self.imed_run_count.get()))
        self.text_exp_loc_label1 = ttk.Label(self, text = 'Exp. folder path') 
        self.text_imed_loc_label1 = ttk.Label(self, text = 'IMED. folder path') 
        self.text_no_exp_label1 = ttk.Label(self, text = 'No. of exp:')
        
    # Buttons
    def create_buttons(self):
    
        self.start_button  = ttk.Button(self, text="Start automated experiments", command=self.start_command)
        self.define_button = ttk.Button(self, text="Define settings", command=self.define_command)
        self.quit_button   = ttk.Button(self, text="Quit", width=8, command=self.destroy)         
        self.stop_button   = ttk.Button(self, text="Stop", width=8, command=self.stop_command)       
        self.reset_button  = ttk.Button(self, text="Reset",width=8, command=self.reset_command)
    
    # Text boxes
    def create_textboxes(self):
        
        self.text_no_exp = tk.Text(self, height=1, width=5) # Text box
        self.text_exp_loc = tk.Text(self, height=1, width=20) # Text box
        self.text_imed_loc = tk.Text(self, height=1, width=20) # Text box
        
    
    # ------------------------------------- PLACE VARIABLES ---------------------------------------------

    def layout_widgets(self):
        
        self.Omni_label1.place(x=400,y=210)
        self.Imed_label1.place(x=500,y=210)
        self.general_label1.place(x=400,y=180)
        self.general_label2.place(x=400,y=240)
        self.Omni_label2.place(x=400,y=270)
        self.Imed_label2.place(x=500,y=270)
        self.general_label3.place(x=150,y=70)
        self.general_label4.place(x=150,y=90)
        self.general_label5.place(x=150,y=180)
        self.text_exp_loc_label1.place(x=50,y=120)
        self.text_imed_loc_label1.place(x=50,y=160)
        self.text_no_exp_label1.place(x=70,y=140)

        self.define_button.place(x=150,y=200)
        
        self.quit_button.place(x=30,y=270)
        self.start_button.place(x=30,y=310)
        
        self.stop_button.place(x=300,y=270)
        self.reset_button.place(x=300,y= 310)

        self.text_no_exp.place(x=150,y=140)
        self.text_exp_loc.place(x=150,y=120)
        self.text_imed_loc.place(x=150,y=160)
        
    # ----------------------------------- FUNCTIONS  -------------------------------------------
    
    def check_if_empty_text(self):
        return [len(i) != 0 for i in [self.text_exp_loc.get("1.0", tk.END).strip(), self.text_imed_loc.get("1.0", tk.END).strip(), self.text_no_exp.get("1.0", tk.END).strip()]]
    
    def set_buttons_enabled(self, enabled: bool):
        # state expects a sequence of flags
        flags = ["!disabled"] if enabled else ["disabled"]
        for btn in (self.start_button, self.define_button, self.quit_button, self.reset_button):
            btn.state(flags)
        
        
    def files_in_folder(self, STAGE, omni_folder, imed_folder):

        # -------------------------------------------------------------------------------
        # Count the number of files in a folder with a specific file names (omni_folder, imed_folder)
        # STAGE: decides whether IMED or OMNI process is active / OMNI or IMED --> determines whether its OMNI or IMED stage /
        # -------------------------------------------------------------------------------
        
        count = 0
        if STAGE == self.OMNI:
            folder = omni_folder
            filename = self.omni_filename
        elif STAGE == self.IMED:
            folder = imed_folder
            filename = self.imed_filename
        else: 
            messagebox.showerror("Error", 'Folder checker error')
            
        with os.scandir(folder) as entries:
            for entry in entries:
                if filename in entry.name:
                    count += 1
        return count
        
    
    def monitoring_folders(self, check_frequency, STAGE, num, omni_folder, imed_folder):
        
        # ------------------------------------------------------------
        # Monitors the number of experiments in a folder
        # ------------------------------------------------------------
        # (1) start monitoring the experiment folder in the automated sequence
        # (2) monitoring omnifolder until number of files equal self.initial_exp_count
        # (3) start monitoring imed folder once omnibus finished (until 1 file is created)
        # (4) monitoring omnifolder until number of files equal 1
         
        # check_frequency: checking the folder in every <check_frequency> seconds
        # STAGE: OMNI or IMED --> determines whether its OMNI or IMED stage
        # ------------------------------------------------------------

        
        if self.process_stop.get():
            return True       
        
        file_count = self.files_in_folder(STAGE, omni_folder, imed_folder) # count number of files in a folder (dependent on STAGE)

        if file_count < num: # --> num
            if STAGE == self.OMNI:
                self.Omni_label1.configure(style="Green.TLabel")
                self.Imed_label1.configure(style="Grey.TLabel")
            elif STAGE == self.IMED:
                self.Omni_label1.configure(style="Grey.TLabel")
                self.Imed_label1.configure(style="Green.TLabel")
                
            self.after(check_frequency * 1000, lambda: self.monitoring_folders(check_frequency,STAGE, num, omni_folder, imed_folder))
            
            return False
            
        else:
            
            self.set_buttons_enabled(False)
            
            if STAGE == self.OMNI:
                
                #messagebox.showerror("Notification", 'Omni finished')
                
                # start IMED
                self.omni_running.set(False)
                self.Omni_label1.configure(style="Grey.TLabel")
                self.imed_running.set(True)
                self.Imed_label1.configure(style="Green.TLabel")
                self.omni_run_count.set(self.omni_run_count.get() + 1)
                self.Omni_label2.config(text=str(self.omni_run_count.get()))
                self.set_buttons_enabled(False)

                if self.imed_run_count.get() == 0:
                    self.monitoring_folders(5, self.IMED, 1, self.omni_folder.get(), self.imed_folder.get())
                else:
                    path = self.imed_folder.get()
                    path_I = os.path.join(self.imed_folder.get(), f"_IMED{self.imed_run_count.get()}")
                    try:
                        os.makedirs(path_I, exist_ok = True)
                    except FileExistsError:
                        pass
    
                    self.monitoring_folders(5, self.IMED, 1, self.omni_folder.get(), path_I)
           
            elif STAGE == self.IMED:
                
                #messagebox.showerror("Notification", 'IMED finished')
                
                self.omni_running.set(False)
                self.Omni_label1.configure(style="Grey.TLabel")
                self.imed_running.set(False)
                self.Imed_label1.configure(style="Grey.TLabel")
                self.imed_run_count.set(self.imed_run_count.get() + 1)
                self.Imed_label2.config(text=str(self.imed_run_count.get()))
                self.set_buttons_enabled(False)
                
                if self.omni_run_count.get() == 0:
                    self.monitoring_folders(5, self.OMNI, self.initial_exp_count.get(), self.omni_folder.get(), self.imed_folder.get())
                else:
                    path = self.imed_folder.get()
                    path_O = os.path.join(self.omni_folder.get(), f"_OMNI{self.imed_run_count.get()}")
                    try:
                        os.makedirs(path_O, exist_ok = True)
                    except FileExistsError:
                        pass
                        
                    self.monitoring_folders(5, self.OMNI, 1, path_O, self.imed_folder)
                
            return True

    # ----------------------------------- BUTTON CALLS  -------------------------------------------
                    
    def start_command(self):

        # ----------------------------------------
        # start the automated OMNIBUS-->IMED loops
        # ----------------------------------------
 
        if not self.variables_set.get():         
            messagebox.showerror("Error", 'Please define variables first!')
            
        elif self.files_in_folder(self.OMNI, self.omni_folder.get(), self.imed_folder.get()) != 0:  
            messagebox.showerror("Error", 'Files \n already in experiment folder')
            
        elif self.files_in_folder(self.IMED, self.omni_folder.get(), self.imed_folder.get()) != 0:    
            messagebox.showerror("Error", 'Files \n already in IMED folder')
        else:

            # START OMNIBUS (actually call folder and app on OMNI computer)
            self.omni_running.set(True)
            self.set_buttons_enabled(False)

            # wait for results
            self.monitoring_folders(5, self.OMNI, self.initial_exp_count.get(), self.omni_folder.get(), self.imed_folder.get())
               
    def stop_command(self):
        # -----------------------------------
        # Terminate the processes (still in progress to complete)
        # -----------------------------------
        
        self.process_stop.set(False)
        if not self.variables_set.get():
            messagebox.showerror("Error", 'Please define variables first!')
        else:
            self.set_buttons_enabled(True)
            # taskkill Application.exe -> cmd
            self.process_stop.set(True)

    def reset_command(self):

        # reset all vairables and widgets
        self.variables_set_message = tk.StringVar(value='')
        self.variables_set = tk.BooleanVar(value = False) 
        self.omni_running = tk.BooleanVar(value = False) 
        self.imed_running = tk.BooleanVar(value = False) 
        self.process_stop = tk.BooleanVar(value = False) 
        self.omni_text = tk.StringVar(value = "OMNIBUS")
        self.initial_exp_count = tk.IntVar(value = 0)
        self.imed_text = tk.StringVar(value = "IMED")
        self.omni_run_count = tk.IntVar(value = 0) 
        self.imed_run_count = tk.IntVar(value = 0)
        self.status_message = tk.StringVar(value='') 
        self.error_message = tk.StringVar(value='') 
        self.omni_folder = tk.StringVar(value = "")
        self.imed_folder = tk.StringVar(value = "")
        self.omni_filename = 'expttsd'
        self.imed_filename = 'imed'
        self.create_textboxes()
        #self.create_buttons()
        #self.layout_widgets()                
        self.create_labels()

    def define_command(self):    
    
        # -------------------------------------------------------------------------------
        # Define initial data for process (file location, initial DoE experiment number)
        # -------------------------------------------------------------------------------
        
        self.status_message.set('')
        self.error_message.set('')
        self.variables_set_message.set('')
        self.variables_set.set(False)
        
        folder_loc      = self.text_exp_loc.get("1.0", tk.END).strip() # Text box
        folder_imed_loc = self.text_imed_loc.get("1.0", tk.END).strip() # Text box
        no_exp          = self.text_no_exp.get("1.0", tk.END).strip() # Text box
    
        if not all( self.check_if_empty_text() ):
            
            self.status_message.set("Please dont leave any brackets empty")
            self.general_label3.config(style="Red.TLabel")
            self.define_button.config(style="Red.TButton")
            
        else:
            if all([os.path.isdir(folder_loc), os.path.isdir(folder_imed_loc)] ):
                self.omni_folder.set(folder_loc) 
                self.imed_folder.set(folder_imed_loc) 
                
                try:
                    no_exp_1 = int(no_exp)
                    if no_exp_1 <= 0:
                        self.error_message.set("Enter a positive integer")
                        self.general_label4.config(style="Red.TLabel")
                        self.define_button.config(style="Red.TButton")
                    else:
                        self.initial_exp_count.set(int(no_exp)) 
                        self.define_button.config(style="Green.TButton")
                        self.variables_set_message.set('Variables set')
                        self.general_label5.config(style="Green.TLabel")
                        self.variables_set.set(True)
                        
                except ValueError:
                    self.error_message.set("Enter a valid integer")
                    self.define_button.config(style="Red.TButton")
            else:
                self.status_message.set("Enter a valid folder path")
                self.define_button.config(style="Red.TButton")  
            

if __name__ == "__main__":
    app = MyApplication()
    app.mainloop()

# C:\Users\user\Desktop\OMNI\1. OMNI
# C:\Users\user\Desktop\OMNI\2. IMED